In [1]:
%load_ext sql
import csv
from tqdm import tqdm

In [2]:
import configparser
import re
import os
import numpy as np
import pandas as pd
import pymssql
import matplotlib.pyplot as plt
from IPython.display import HTML
config = configparser.ConfigParser()
config.read("dm_aou.ini")
properties = config.defaults()
db_user, db_pass = properties['user'],properties['password']
database = 'ohdsi_cumc_hpo'
schema = 'dbo'
conn = pymssql.connect("aousql.dbmi.columbia.edu", db_user, db_pass, database)

In [3]:
import glob
import json

In [4]:
root_path = '/home/dbmi/xj2193/HPO/aou_ehr_validator/resources/omop'
all_json = glob.glob(f'{root_path}/*.json', recursive=True)

In [5]:
omop_files = {}
pattern = "omop/(.*?).json"
file_name_list = ['care_site', 'condition_occurrence', 'death', 'device_exposure', 'drug_exposure',
              'fact_relationship', 'location', 'measurement', 'note', 'observation', 
              'procedure_occurrence', 'visit_occurrence', 'provider', 'pii_address', 'pii_email', 
              'pii_name', 'pii_mrn', 'pii_phone_number', 'specimen', 'participant_match']
for file_path in all_json:
    file_name = re.search(pattern, file_path).group(1)
    #file_name_list.append(file_name)
    with open(file_path) as file:
        content = json.load(file)
        col_list = {}
        for col in content:
            col_list[col['name']] = col['type']
        omop_files[file_name] = col_list

In [6]:
from datetime import date, datetime, tzinfo
from dateutil import tz

NYC = tz.gettz('America/New_York')

In [7]:
def change_col_type(omop_check_files, df, file_name):
    for col in df.columns:
        col_type = omop_check_files[file_name][col]
        if col_type == 'string':
            df[col] = df[col].fillna('')
            df[col] = df[col].astype(str)
            df[col] = df[col].apply(lambda x: x.replace('"', '""').replace('\n', ' '))
        elif col_type == 'integer':
            df[col] = df[col].astype('Int64')
        elif col_type == 'float':
            df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)
        elif col_type == 'date':
            df[col] = df[col].apply(lambda x: x.isoformat() if pd.notnull(x) else x)
        elif col_type == 'timestamp':
            df[col] = df[col].astype(object).where(df[col].notnull(), None)
            df[col] = df[col].apply(lambda x: x.replace(tzinfo=NYC).isoformat() if pd.notnull(x) else x)
        else:
            pass
    df = df.replace(np.nan, '', regex=True)
    return df

### Extract each OMOP table

In [8]:
query_path = '/home/dbmi/xj2193/HPO/omop_table_sql'
output_path = '/home/dbmi/xj2193/HPO/HPO_submission/'

In [9]:
delimiter_hpo = ','
quotechar_hpo = '"'

In [10]:
person_query = open(os.path.join(query_path, 'person.sql'), 'r')
table_name = 'person'
person_query_script = person_query.read().format(database, schema)
person_table = pd.read_sql(person_query_script, conn)

person_list = person_table['person_id'].to_list()
person_list = str(person_list).replace('[', '').replace(']', '')

In [11]:
###Export person table
is_header_added = False
with open(output_path + 'person.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=delimiter_hpo, quotechar=quotechar_hpo, quoting=csv.QUOTE_ALL)
    person_table = change_col_type(omop_files, person_table, 'person')
    if not is_header_added:
        csv_writer.writerow(person_table.columns)
        is_header_added = True
    for row in tqdm(person_table.itertuples(index=False), total = person_table.shape[0]):
        csv_writer.writerow(row)

100%|██████████| 10923/10923 [00:00<00:00, 128146.74it/s]


In [14]:
empty_table_list = ['care_site', 'provider', 'fact_relationship', 'pii_address', 'pii_email', 'pii_mrn', 'specimen']
pii_table_list = ['pii_name', 'pii_phone_number', 'participant_match']
def export_to_csv(file_path, query, conn, omop_check_files, file_name, empty_list):
    print('Start exporting {}...'.format(file_name))
    if file_name in empty_list:
        data = pd.read_sql(query, conn)
        with open(file_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile, delimiter=delimiter_hpo, quotechar=quotechar_hpo, quoting=csv.QUOTE_ALL)
            csv_writer.writerow(data.columns)
    elif file_name in pii_table_list:
        data = pd.read_sql(query, conn)
        is_header_added = False
        with open(file_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile, delimiter=delimiter_hpo, quotechar=quotechar_hpo, quoting=csv.QUOTE_ALL)
            if not is_header_added:
                csv_writer.writerow(data.columns)
                is_header_added = True
            for row in tqdm(data.itertuples(index=False), total = data.shape[0]):
                csv_writer.writerow(row)
    else:
        data = pd.read_sql(query, conn, chunksize=50000)
        is_header_added = False
        with open(file_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile, delimiter=delimiter_hpo, quotechar=quotechar_hpo, quoting=csv.QUOTE_ALL)
            for batch in data:
                if file_name in file_name_list:
                    batch = change_col_type(omop_check_files, batch, file_name)
                else:
                    pass
                if not is_header_added:
                    csv_writer.writerow(batch.columns)
                    is_header_added = True
                for row in tqdm(batch.itertuples(index=False), total=len(batch)):
                    csv_writer.writerow(row)

In [15]:
def export_omop_file(file_name, query_path, output_path, connection, omop_check_files, empty_list):
    query = open(os.path.join(query_path, file_name + '.sql'), 'r')
    query_script = query.read()
    if 'person_id' in query_script:
        query_script = query_script.format(database, schema, person_list)
    else:
        query_script = query_script.format(database, schema)
    output_file_path = output_path + file_name + '.csv'
    export_to_csv(output_file_path, query_script, connection, omop_check_files,file_name, empty_list)
    query.close()
    return file_name + '.csv file exported'

In [16]:
for file in file_name_list:
    export_omop_file(file, query_path, output_path, conn, omop_files, empty_table_list)

Start exporting care_site...
Start exporting condition_occurrence...


100%|██████████| 26569/26569 [00:00<00:00, 138195.62it/s]


Start exporting death...


100%|██████████| 64/64 [00:00<00:00, 148553.10it/s]


Start exporting device_exposure...


...

100%|██████████| 12668/12668 [00:00<00:00, 426165.35it/s]


Start exporting pii_mrn...
Start exporting pii_phone_number...


100%|██████████| 12668/12668 [00:00<00:00, 358095.16it/s]


Start exporting specimen...
Start exporting participant_match...


100%|██████████| 10924/10924 [00:00<00:00, 335269.33it/s]
